\
            # 43. 前端基础：JavaScript（JS）（JavaScript Fundamentals）

            目标：用“够用且不易踩坑”的方式掌握 JS 的核心语法与浏览器运行模型。
本章用可运行的浏览器演示（Jupyter 输出区执行 HTML/JS）。
如果你的 Notebook 环境禁用了脚本执行，也可以把示例 HTML 复制到浏览器打开。

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- 基本编程概念（变量/条件/循环/函数）
- 了解 HTML（标签/属性）更佳


## 知识点地图

- 1. 运行环境与最小心智模型：JS 在哪里跑？
- 2. 变量与类型：let/const/var、基本类型、===
- 3. 函数与作用域：闭包、this、箭头函数
- 4. 对象与数组：解构、展开、常用高阶方法
- 5. 异步：Promise 与 async/await（串行、并行、错误处理）
- 6. 模块化与工程化（了解）：import/export、npm、构建工具


## 自检清单（学完打勾）

- [ ] 能正确使用 let/const，避免 var/提升带来的坑
- [ ] 理解基本类型与引用类型，知道 ===/== 的差异
- [ ] 会写函数/箭头函数，理解闭包与 this
- [ ] 会操作数组/对象（解构、展开、map/filter/reduce）
- [ ] 理解 Promise 与 async/await 的错误处理与串并行


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：运行环境与最小心智模型：JS 在哪里跑？

- JS 可以运行在 **浏览器**（DOM/Web API）和 **Node.js**（文件/网络等）。
- 浏览器里：JS + Web API + 事件循环（event loop）共同工作。
- 你写的 JS 不是“随时能阻塞”的：长时间同步计算会卡 UI。

下面用 Jupyter 输出区渲染一段 HTML，让你看到 JS 的最小执行效果。


In [ ]:
from IPython.display import HTML, display

display(HTML("""
<div id="js-demo-1" style="padding:8px;border:1px solid #ddd;">
  <div><b>JS Demo</b>: <span id="txt">(waiting...)</span></div>
  <button id="btn">Click me</button>
</div>
<script>
  const txt = document.getElementById('txt');
  const btn = document.getElementById('btn');
  txt.textContent = 'ready';
  btn.addEventListener('click', () => {
    txt.textContent = 'clicked at ' + new Date().toLocaleTimeString();
  });
</script>
"""))


## 知识点 2：变量与类型：let/const/var、基本类型、===

- 推荐：
  - 默认用 `const`（引用本身不变）。
  - 需要重新赋值时用 `let`。
- 避免：`var`（函数作用域 + 变量提升 + 容易引入隐蔽 bug）。
- 基本类型：`number`/`string`/`boolean`/`null`/`undefined`/`symbol`/`bigint`。
- 引用类型：`object`（对象/数组/函数/日期等）。
- 比较：优先使用 `===`（严格相等），不要依赖 `==` 的隐式类型转换。

常见坑：
- `typeof null === 'object'`（历史遗留）。
- `NaN !== NaN`，判断用 `Number.isNaN(x)`。


In [ ]:
from IPython.display import HTML, display

code = r"""
<script>
  const lines = [];
  lines.push('typeof null -> ' + (typeof null));
  lines.push('NaN === NaN -> ' + (NaN === NaN));
  lines.push('Number.isNaN(NaN) -> ' + (Number.isNaN(NaN)));
  lines.push("'1' == 1 -> " + ('1' == 1));
  lines.push("'1' === 1 -> " + ('1' === 1));
  document.getElementById('js-demo-2').textContent = lines.join('\n');
</script>
"""

display(HTML("""
<pre id="js-demo-2" style="padding:8px;border:1px solid #ddd;"></pre>
""" + code))


## 知识点 3：函数与作用域：闭包、this、箭头函数

- 函数定义方式：`function f(){}`、函数表达式、箭头函数 `() => {}`。
- 闭包：函数“记住”它创建时的外部变量（常用于工厂函数、回调、封装私有状态）。
- `this`：取决于调用方式（方法调用/普通函数/构造函数/显式绑定）。
- 箭头函数没有自己的 `this`，会捕获外层 `this`（写回调时更稳定）。

实践建议：
- 写回调优先箭头函数；
- 写对象方法可用普通函数（如果需要动态 this）。


In [ ]:
from IPython.display import HTML, display

display(HTML(r"""
<pre id="js-demo-3" style="padding:8px;border:1px solid #ddd;"></pre>
<script>
  const out = [];

  function makeAdder(x) {
    return function(y){ return x + y; };
  }
  const add10 = makeAdder(10);
  out.push('closure add10(3) = ' + add10(3));

  const obj = {
    n: 1,
    incLater() {
      setTimeout(function(){ out.push('function this.n = ' + this.n); render(); }, 10);
      setTimeout(() => { out.push('arrow this.n = ' + this.n); render(); }, 20);
    }
  };

  function render(){
    document.getElementById('js-demo-3').textContent = out.join('\n');
  }

  render();
  obj.incLater();
</script>
"""))


## 知识点 4：对象与数组：解构、展开、常用高阶方法

- 对象/数组是前端数据结构的主力。
- 解构：快速取值并赋默认值。
- 展开：`{...obj}` / `[...arr]` 进行浅拷贝与合并。
- 常用数组方法：`map/filter/reduce/find/some/every`。

注意：展开/浅拷贝不等于深拷贝；深层对象仍共享引用。


In [ ]:
from IPython.display import HTML, display

display(HTML(r"""
<pre id="js-demo-4" style="padding:8px;border:1px solid #ddd;"></pre>
<script>
  const out = [];
  const user = {id: 1, profile: {name: 'Alice'}, age: 18};
  const {id, age=0} = user;
  out.push('id=' + id + ', age=' + age);

  const a = [1,2,3];
  const b = [...a, 4];
  out.push('b=' + JSON.stringify(b));

  const nums = [1,2,3,4,5];
  const evens = nums.filter(x => x%2===0).map(x => x*x);
  out.push('evens squared=' + JSON.stringify(evens));

  document.getElementById('js-demo-4').textContent = out.join('\n');
</script>
"""))


## 知识点 5：异步：Promise 与 async/await（串行、并行、错误处理）

- JS 的 I/O（网络/定时器/事件）通常是异步的。
- Promise 表示“未来会完成的结果”。
- `async/await` 是 Promise 的语法糖，让异步代码像同步一样读。

关键点：
- 串行：`await a(); await b();`（后一个依赖前一个的结果）
- 并行：`await Promise.all([a(), b()])`（互不依赖，提升吞吐）
- 错误处理：`try/catch` + 对失败策略做清晰定义（重试、降级、返回错误）。


In [ ]:
from IPython.display import HTML, display

display(HTML(r"""
<pre id="js-demo-5" style="padding:8px;border:1px solid #ddd;"></pre>
<script>
  const out = [];
  const sleep = (ms) => new Promise(res => setTimeout(res, ms));

  async function job(name, ms, fail=false){
    await sleep(ms);
    if (fail) throw new Error('fail ' + name);
    return name + ':ok';
  }

  async function main(){
    try {
      const r1 = await job('a', 30);
      const r2 = await job('b', 10);
      out.push('serial -> ' + [r1, r2].join(', '));

      const rs = await Promise.all([job('c', 20), job('d', 15)]);
      out.push('parallel -> ' + rs.join(', '));

      await job('e', 5, true);
    } catch (e) {
      out.push('caught -> ' + e.message);
    } finally {
      document.getElementById('js-demo-5').textContent = out.join('\n');
    }
  }

  main();
</script>
"""))


## 知识点 6：模块化与工程化（了解）：import/export、npm、构建工具

- ES Module：`export`/`import` 把代码拆成可复用模块。
- npm 管理依赖；构建工具（Vite/Webpack/Rollup）处理打包、转译、压缩。
- 现代前端常见搭配：TypeScript（类型）+ ESLint/Prettier（规范）+ 单元测试。

学习路径建议：
1) 先熟 JS 语法与异步 2) 再学框架（Vue/React） 3) 最后补工程化细节。


## 常见坑

- 用 var 导致提升与作用域混乱
- 混用 == 与 ===，依赖隐式类型转换
- 误解 this：回调里 this 丢失（用箭头函数或 bind）
- 浅拷贝当深拷贝：对象嵌套引用导致“改一处动全局”
- Promise 并发写成串行：本可 Promise.all 却逐个 await


## 综合小案例：纯 JS Todo：增删改查 + localStorage 持久化

实现一个最小 Todo：
- 添加/完成/删除
- 列表渲染
- localStorage 保存与恢复

提示：这里用“单文件组件”思路：state + render + event handlers。


In [ ]:
from IPython.display import HTML, display

html = r"""
<div id="todo-app" style="font-family:system-ui;padding:10px;border:1px solid #ddd;max-width:520px;">
  <h3 style="margin:0 0 8px 0;">Todo (Pure JS)</h3>
  <div style="display:flex;gap:6px;">
    <input id="todo-input" placeholder="new todo" style="flex:1;padding:6px;" />
    <button id="todo-add">Add</button>
  </div>
  <ul id="todo-list" style="padding-left:18px;"></ul>
</div>
<script>
  const KEY = 'demo.todos.v1';
  let state = { todos: [] };

  function load(){
    try { state.todos = JSON.parse(localStorage.getItem(KEY) || '[]'); }
    catch { state.todos = []; }
  }
  function save(){ localStorage.setItem(KEY, JSON.stringify(state.todos)); }

  function addTodo(title){
    state.todos.push({id: Date.now().toString(36), title, done:false});
    save(); render();
  }
  function toggle(id){
    const t = state.todos.find(x => x.id === id);
    if (t) t.done = !t.done;
    save(); render();
  }
  function removeTodo(id){
    state.todos = state.todos.filter(x => x.id !== id);
    save(); render();
  }

  function render(){
    const ul = document.getElementById('todo-list');
    ul.innerHTML = '';
    for (const t of state.todos) {
      const li = document.createElement('li');
      li.style.margin = '6px 0';
      li.innerHTML = `
        <label style="cursor:pointer;">
          <input type="checkbox" ${t.done ? 'checked' : ''} />
          <span style="${t.done ? 'text-decoration:line-through;color:#777' : ''}">${t.title}</span>
        </label>
        <button style="margin-left:8px;">Del</button>
      `;
      li.querySelector('input').addEventListener('change', () => toggle(t.id));
      li.querySelector('button').addEventListener('click', () => removeTodo(t.id));
      ul.appendChild(li);
    }
  }

  load();
  render();

  document.getElementById('todo-add').addEventListener('click', () => {
    const inp = document.getElementById('todo-input');
    const title = (inp.value || '').trim();
    if (!title) return;
    inp.value = '';
    addTodo(title);
  });
</script>
"""

display(HTML(html))


## 自测题（不写代码也能回答）

- 为什么推荐 const/let，而不推荐 var？
- === 与 == 的差异是什么？为什么前端工程里通常禁用 ==？
- 闭包是什么？它在前端里最常见的使用场景有哪些？
- Promise.all 适合什么场景？失败策略怎么设计？


## 练习题（建议写代码）

- 为 Todo 增加过滤：all/active/done。
- 为 Todo 增加编辑功能：双击进入编辑，回车保存。
- 加入“防抖/节流”示例：输入时实时搜索（了解）。
